In [1]:
import numpy as np
from sklearn.decomposition import PCA
from sklearn.model_selection import RepeatedKFold
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
import os
import h5py
import pandas as pd
from multiprocessing.dummy import Pool as ThreadPool
import time

def opti_func(train_i,test_i,crt_clf,X,Y):
    filt,clf = crt_clf()
    
    X_train, X_test = X[train_i], X[test_i]
    y_train, y_test = Y[train_i], Y[test_i]
    
    X_train = filt.fit_transform(X_train, y_train)
    X_test = filt.transform(X_test)
    clf.fit(X_train, y_train)
    
    return clf.score(X_test,y_test)*100
    
    
data_path = os.path.join('.','tests','test_output','data.hdf5')

dat = h5py.File(data_path, 'r')

X = np.array(dat.get('data'))
Y = np.array(dat.get('data_label'))
dat.close()

# Number of features to take 

featurenum_list = [10, 100, 1000, 10000, 100000] 
Ks = [1, 3, 5, 11]

In [ ]:
#### Decision Tree ####
start_time = time.time()
for n in featurenum_list: 
    rkf = RepeatedKFold(n_splits=10, n_repeats=10)
    crt_clsf = lambda a_1 = chi2, a_2 = n :(SelectKBest(a_1, k=a_2),DecisionTreeClassifier())
    pool = ThreadPool(100)
    acc = [pool.apply(opti_func,args=(i_train,i_test,crt_clsf,X,Y)) for i_train,i_test in rkf.split(X)]
    print("For SELKB+DT n = {}, mean = {}, var = {}".format(n,np.mean(acc),np.var(acc)))
    print("--- %s seconds ---" % (time.time() - start_time))

For SELKB+DT n = 10, mean = 8.4225, var = 0.22920208333333336
--- 10726.920655965805 seconds ---
For SELKB+DT n = 100, mean = 13.343749999999998, var = 0.30522135416666657
--- 21289.44335603714 seconds ---
For SELKB+DT n = 1000, mean = 32.75333333333333, var = 1.105891666666666
--- 32194.008802890778 seconds ---
For SELKB+DT n = 10000, mean = 68.40624999999999, var = 1.666679687500001
--- 44851.85349535942 seconds ---
For SELKB+DT n = 100000, mean = 73.8375, var = 0.9233506944444432
--- 82652.74457716942 seconds ---


In [ ]:
for n in featurenum_list: 
    rkf = RepeatedKFold(n_splits=10, n_repeats=1)
    crt_clsf = lambda a_1 = n :(PCA(n_components=a_1),DecisionTreeClassifier())
    pool = ThreadPool(100)
    acc = [pool.apply(opti_func,args=(i_train,i_test,crt_clsf,X,Y)) for i_train,i_test in rkf.split(X)]
    print("For PCA+DT n = {}, mean = {}, var = {}".format(n,np.mean(acc),np.var(acc)))
    print("--- %s seconds ---" % (time.time() - start_time))

For PCA+DT n = 10, mean = 65.13083333333333, var = 0.9046534722222239
--- 92040.20052218437 seconds ---
For PCA+DT n = 100, mean = 68.57666666666667, var = 0.7059277777777795
--- 103489.49698448181 seconds ---
For PCA+DT n = 1000, mean = 64.98583333333333, var = 1.1312576388888909
--- 141887.24352002144 seconds ---


In [ ]:
#### KNN ####
for n in featurenum_list:
    for k in Ks:
        rkf = RepeatedKFold(n_splits=10, n_repeats=10)
        crt_clsf = lambda a_1 = chi2, a_2 = n, a_3 = k:(SelectKBest(a_1, k=a_2),KNeighborsClassifier(n_neighbors=a_3))
        pool = ThreadPool(100)
        acc = [pool.apply(opti_func,args=(i_train,i_test,crt_clsf,X,Y)) for i_train,i_test in rkf.split(X)]
        print("For SELKB+knn n = {}, k = {}, mean = {}, var = {}".format(n,k,np.mean(acc),np.var(acc)))
        print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
for n in featurenum_list:
    for k in Ks:
        rkf = RepeatedKFold(n_splits=10, n_repeats=10)
        crt_clsf = lambda a_1 = n, a_3 = k :(PCA(n_components=a_1),KNeighborsClassifier(n_neighbors=a_3))
        pool = ThreadPool(100)
        acc = [pool.apply(opti_func,args=(i_train,i_test,crt_clsf,X,Y)) for i_train,i_test in rkf.split(X)]
        print("For PCA+knn n = {}, k = {}, mean = {}, var = {}".format(n,k,np.mean(acc),np.var(acc)))
        print("--- %s seconds ---" % (time.time() - start_time))